<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

# Project Instructions
- Split the data into three, one with information about the client, another containing campaign data, and a third to store information about economics at the time of the campaign.
- Rename columns, including "client_id" to "id" in client (leave as-is in the other subsets); "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign; and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.
- Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
- Remove periods from the "job" column.
- Convert "previous_outcome" and "campaign_outcome" values to binary (1 or 0), including the conversion of "nonexistent" to 0.
- Add a campaign_id as the first column in campaign, where all rows have a value of 1.
- Create a datetime column called last_contact_date, in the format of "month-day-year", where the year is 2022, and the month and day values are taken from the "month" and "day_of_week" columns.
- Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.
- Save the three subsets to csv files without an index.
- Create a dictionary called database_design using the three table names as keys and assigning values as the respective subset DataFrames you have created.
- Print database_design to check your solution.

In [44]:
# Start coding...
import pandas as pd
import numpy as np

In [45]:
# Store and print database_design
database_design = pd.read_csv("bank_marketing.csv")

print(database_design)

       client_id  age          job  ... euribor3m nr_employed    y
0              0   56    housemaid  ...     4.857      5191.0   no
1              1   57     services  ...     4.857      5191.0   no
2              2   37     services  ...     4.857      5191.0   no
3              3   40       admin.  ...     4.857      5191.0   no
4              4   56     services  ...     4.857      5191.0   no
...          ...  ...          ...  ...       ...         ...  ...
41183      41183   73      retired  ...     1.028      4963.6  yes
41184      41184   46  blue-collar  ...     1.028      4963.6   no
41185      41185   56      retired  ...     1.028      4963.6   no
41186      41186   44   technician  ...     1.028      4963.6  yes
41187      41187   74      retired  ...     1.028      4963.6   no

[41188 rows x 22 columns]


In [46]:
database_design.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [47]:
database_design.rename(columns= {'client_id':'id', 'duration': 'contact_duration', 
                                 'previous': 'previous_campaign_contacts',    'y':'campaign_outcome','poutcome':'previous_outcome',
                                 'campaign':'number_contacts', 'euribor3m':'euribor_three_months',
                                 'nr_employed':'number_employed'}, inplace = True)

In [48]:
database_design.columns

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan', 'contact', 'month', 'day_of_week', 'contact_duration',
       'number_contacts', 'pdays', 'previous_campaign_contacts',
       'previous_outcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx',
       'euribor_three_months', 'number_employed', 'campaign_outcome'],
      dtype='object')

In [49]:
database_design.education.value_counts()

university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

In [50]:
database_design.education = database_design.education.str.replace('.','_')
database_design.education = database_design.education.replace('unknown', np.nan)
database_design.education.value_counts()

university_degree      12168
high_school             9515
basic_9y                6045
professional_course     5243
basic_4y                4176
basic_6y                2292
illiterate                18
Name: education, dtype: int64

In [51]:
database_design.job.value_counts()

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [52]:
database_design.job = database_design.job.str.replace('.','')
database_design.job.value_counts()

admin            10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [53]:
database_design[['previous_outcome','campaign_outcome']].value_counts()

previous_outcome  campaign_outcome
nonexistent       no                  32422
failure           no                   3647
nonexistent       yes                  3141
success           yes                   894
failure           yes                   605
success           no                    479
dtype: int64

In [54]:
database_design.previous_outcome = database_design.previous_outcome.replace({'success':1, 'failure':0, 'nonexistent':0})
database_design.campaign_outcome = database_design.campaign_outcome.replace({'yes':1, 'no':0})
database_design[['previous_outcome','campaign_outcome']].value_counts()

previous_outcome  campaign_outcome
0                 0                   36069
                  1                    3746
1                 1                     894
                  0                     479
dtype: int64

In [55]:
database_design['campaign_id'] = 1
database_design['last_contact_date'] = database_design.month + '-' + database_design.day_of_week + '-2022'
database_design.last_contact_date
database_design.last_contact_date = pd.to_datetime(database_design["last_contact_date"], format="%b-%a-%Y")

In [56]:
client = database_design[['id','age','job','marital','education','credit_default','housing','loan']]

campaign = database_design[['campaign_id','id','number_contacts','contact_duration','pdays','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']]
campaign.rename(columns = {'id':'client_id'}, inplace=True)

economics = database_design[['id','emp_var_rate','cons_price_idx','euribor_three_months','number_employed']]
economics.rename(columns = {'id': 'client_id'}, inplace=True)

In [57]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

In [58]:
database_design = {'client': client, 'campaign': campaign, 'economics': economics}
database_design

{'client':           id  age          job  ... credit_default housing loan
 0          0   56    housemaid  ...             no      no   no
 1          1   57     services  ...        unknown      no   no
 2          2   37     services  ...             no     yes   no
 3          3   40        admin  ...             no      no   no
 4          4   56     services  ...             no      no  yes
 ...      ...  ...          ...  ...            ...     ...  ...
 41183  41183   73      retired  ...             no     yes   no
 41184  41184   46  blue-collar  ...             no      no   no
 41185  41185   56      retired  ...             no     yes   no
 41186  41186   44   technician  ...             no      no   no
 41187  41187   74      retired  ...             no     yes   no
 
 [41188 rows x 8 columns],
 'campaign':        campaign_id  client_id  ...  campaign_outcome  last_contact_date
 0                1          0  ...                 0         2022-05-01
 1                1    